# Donnée Static Grid Model

1. Accès au dataset des lignes françaises du GRT RTE en date de 2019. Données en accès : I_max saisonniers, stations, longueurs, caractéristiques
Source : https://www.services-rte.com/en/learn-more-about-our-services/static-grid-model.html
Problèmes éventuels : données lacunaire et datées (2019)

2. Dataset : [Lignes aériennes RTE – nouveau découpage (au 10 juin 2024)](https://odre.opendatasoft.com/explore/dataset/lignes-aeriennes-rte-nv/information/?disjunctive.etat&disjunctive.tension&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6ImxvbmdpdHVkZV9kZWJ1dF9zZWdtZW50Iiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoicmFuZ2UtQWNjZW50In1dLCJ4QXhpcyI6InRlbnNpb24iLCJtYXhwb2ludHMiOm51bGwsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJldGF0Iiwic2VyaWVzQnJlYWtkb3duVGltZXNjYWxlIjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJsaWduZXMtYWVyaWVubmVzLXJ0ZS1udiIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuZXRhdCI6dHJ1ZSwiZGlzanVuY3RpdmUudGVuc2lvbiI6dHJ1ZX19fV0sInRpbWVzY2FsZSI6IiIsImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9&location=13,45.56282,4.83351&basemap=jawg.light) / 

## Importations de librairie

In [2]:
import pandas as pd
import plotly.express as px
 

## DATA Lignes

### Ligne 2019 + caractéristiques

In [3]:
path = '/Users/matthiasmolenat/repos/congestion/data/RT_ligne_2019.csv'
df_ligne_RT_2019 = pd.read_csv(path,delimiter=';',index_col=False)
df_ligne_RT_2019['P_max_summer (MW)']=df_ligne_RT_2019['Voltage level (kV)']*df_ligne_RT_2019['I_max_Summer']/1000
df_ligne_RT_2019.head()

,ref,Circuit name,sub_1_short,sub_2_short,Voltage level (kV),I_max_Summer,I_max_Interseason_1,I_max_Interseason_2,I_max_Winter,Resistance R (Ω),Reactance X (Ω),Capacity (nF),Half-Susceptance Hd/2 (μS),Length (m),Unnamed: 14,P_max_summer (MW)
0,AIRVAL61BONN6,LIT 225kV N0 1 AIRVAULT-BONNEAU,AIRVA,BONN6,225,932.0,932.0,1145.0,1145.0,"3,064","20,350","762,736","119,810",52792,NaN,209.700
1,AIRVAL61JUMEA,LIT 225kV N0 1 AIRVAULT-JUMEAUX (LES),AIRVA,JUMEA,225,1249.0,1337.0,1337.0,1434.0,"0,330","2,212","50,501","7,933",5507,NaN,281.025
2,ALBERL61CHAV6,LIT 225kV N0 1 ALBERTVILLE-CHAVANOD,ALBER,CHAV6,225,617.0,685.0,685.0,742.0,"4,460","17,210","17,210","60,000",42135,NaN,138.825
3,ALBERL61CONTA,LIT 225kV N0 1 ALBERTVILLE-CONTAMINE,ALBER,CONTA,225,565.0,673.0,673.0,765.0,"2,352","10,043","225,097","35,358",24925,NaN,127.125
4,ALBERL61LONG6,LIT 225kV N0 1 ALBERTVILLE-LONGEFAN,ALBER,LONG6,225,565.0,673.0,673.0,765.0,"4,486","17,749","391,180","61,446",43669,NaN,127.125


In [4]:
df_ligne_RT_2019.shape

(921, 16)

#### Chercher toutes les lignes associées à un poste

In [5]:
name_str = 'ECHALAS'

df_ligne_RT_2019[df_ligne_RT_2019.apply(lambda row: row.astype(str).str.contains(name_str).any(), axis=1)]


,ref,Circuit name,sub_1_short,sub_2_short,Voltage level (kV),I_max_Summer,I_max_Interseason_1,I_max_Interseason_2,I_max_Winter,Resistance R (Ω),Reactance X (Ω),Capacity (nF),Half-Susceptance Hd/2 (μS),Length (m),Unnamed: 14,P_max_summer (MW)
225,ECHALL61GIVOR,LIT 225kV N0 1 ECHALAS - GIVORS,ECHAL,GIVOR,225,900.0,910.0,1015.0,1080.0,"0,420","2,890","118,610","18,000",6946,NaN,202.500
226,ECHALL61RIVIE,LIT 225kV N0 1 ECHALAS-RIVIERE (LA),ECHAL,RIVIE,225,1800.0,2030.0,2030.0,2224.0,"0,865","8,422","363,397","57,082",28844,NaN,405.000
227,ECHALL61SOLEI,LIT 225kV N0 1 ECHALAS-SOLEIL (LE),ECHAL,SOLEI,225,1375.0,1437.0,1437.0,1500.0,"0,890","8,830","263,939","54,000",28207,NaN,309.375
515,ECHALL62GIVOR,LIT 225kV N0 2 ECHALAS - GIVORS,ECHAL,GIVOR,225,900.0,910.0,1015.0,1080.0,"0,420","2,890","140,056","22,000",7034,NaN,202.500
650,CHARPL71ECHAL,LIT 400KV N0 1 CHARPENAY - ECHALAS,CHARP,ECHAL,400,NaN,NaN,NaN,NaN,"0,526","9,604","410,796","64,528",32890,NaN,NaN
693,ECHALL71P.COR,LIT 400KV N0 1 ECHALAS - PIVOZ-CORDIER,ECHAL,P.COR,400,NaN,NaN,NaN,NaN,"0,268","4,890","209,170","32,856",16747,NaN,NaN
828,CHARPL72ECHAL,LIT 400kV N0 2 CHARPENAY-ECHALAS,CHARP,ECHAL,400,3465.0,3622.0,3622.0,3780.0,"0,533","9,712","417,219","65,537",32882,NaN,1386.000
859,ECHALL72P.COR,LIT 400kV N0 2 ECHALAS-PIVOZ-CORDIER,ECHAL,P.COR,400,2170.0,2300.0,2300.0,2400.0,"0,268","4,901","208,756","32,791",88692,NaN,868.000


### Lignes 2024 

In [6]:
path_ligne_aer = '/Users/matthiasmolenat/repos/congestion/data/lignes-aeriennes-rte-nv.csv'
df_ligne_RT_2023 = pd.read_csv(path_ligne_aer,delimiter=';')
df_ligne_RT_2023.head()

,Type ouvrage,Code ligne 1,Nom ligne 1,Propriétaire ligne 1,Etat,TENSION,Source donnée,Geo Shape,Nombre circuit,Code ligne 2,...,Code ligne 3,Nom ligne 3,Propriétaire ligne 3,Code ligne 4,Nom ligne 4,Propriétaire ligne 4,Code ligne 5,Nom ligne 5,Proprietaire ligne 5,geo_point_2d
0,AERIEN,ROLAML61ZRIGO,LIAISON 225kV N0 1 PUSY - RIGOTTE (LA) - ROLAM...,MAINTENU PAR ET PROPRIETE RTE,EN EXPLOITATION,225kV,RTE,"{""coordinates"": [[5.7085749089, 47.7587819641]...",1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"47.82855513836, 5.491070148827368"
1,AERIEN,CEBAZL31RIOM,LIAISON 63kV N0 1 CEBAZAT-RIOM,MAINTENU PAR ET PROPRIETE RTE,EN EXPLOITATION,63kV,RTE,"{""coordinates"": [[3.1228486356, 45.8837232437]...",1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.883755153600006, 3.12276277745"
2,AERIEN,CROIXL62P.ME5,LIAISON 225kV N0 2 CROIX (LA) - PETIT-MENIN,MAINTENU PAR ET PROPRIETE RTE,EN EXPLOITATION,225kV,RTE,"{""coordinates"": [[3.1302285506, 50.7103537288]...",1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50.71040177335, 3.1305627690499995"
3,AERIEN,COULAL71TAVEL,LIAISON 400kV N0 1 COULANGE - TAVEL,MAINTENU PAR ET PROPRIETE RTE,EN EXPLOITATION,400kV,RTE,"{""coordinates"": [[4.7217727214, 44.6325603872]...",1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"44.632447227149996, 4.72240015955"
4,AERIEN,ARGE8L41GACE,LIAISON 90kV N0 1 ARGENTAN-GACE,MAINTENU PAR ET PROPRIETE RTE,EN EXPLOITATION,90kV,RTE,"{""coordinates"": [[-0.0089275157, 48.7425637902...",2,ARGE8L41VASTO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.77603848796921, 0.01729029857262007"


In [7]:
for i in range(10):
    print(df_ligne_RT_2023['Geo Shape'].iloc[i])

{"coordinates": [[5.7085749089, 47.7587819641], [5.7052874722, 47.7597415148], [5.7020076415, 47.7606943545], [5.698757684, 47.7616357054], [5.6953049994, 47.7626365488], [5.6918384105, 47.7636537986], [5.6885975102, 47.7645936958], [5.6854608366, 47.7655113296], [5.6821507922, 47.766463379], [5.6788886524, 47.7674125155], [5.6757950979, 47.7683181097], [5.6726081466, 47.7692518574], [5.6694206248, 47.7701782383], [5.6662386861, 47.7710846561], [5.6629626419, 47.7720336368], [5.6595318128, 47.7730436453], [5.6571071307, 47.773739896], [5.6540862737, 47.774640569], [5.6512171272, 47.7755088506], [5.6485998803, 47.7763109406], [5.6455207428, 47.7772279806], [5.642128742, 47.7782519257], [5.6387997672, 47.77925634], [5.6354268589, 47.7802670284], [5.6321643654, 47.7812427192], [5.6287007417, 47.7822868256], [5.6253284113, 47.7832890925], [5.6219103506, 47.7843139691], [5.6187940086, 47.7852526569], [5.6154170236, 47.7862502552], [5.6119152251, 47.7872919455], [5.6084198118, 47.7883288459]

In [8]:
df_ligne_RT_2023.shape

(13210, 22)

## DATA Postes 

1.

## DATA Contraintes 

1. [Contraintes résiduelles prospectives sur le RPT à horizon 3 à 5 ans (à février 2024)](https://odre.opendatasoft.com/explore/dataset/contraintes-region/export/?location=7,46.18744,3.57056&basemap=jawg.light)

In [9]:
path = '/Users/matthiasmolenat/repos/congestion/data/contraintes-region.csv'
df_contraintes_sites = pd.read_csv(path,delimiter=';',index_col=False)
df_contraintes_sites.head()

,Région,Ouvrage,Nom de l'ouvrage,Puissance max de l'ouvrage,Poste 1,Pourcentage 1,Poste 2,Pourcentage 2,Poste 3,Pourcentage 3,geom,centroid
0,AUVERGNE-RHÔNE-ALPES,SSSAUL31ZSSRO,LIAISON 63kV N0 1 ENVAL-ST-PIERRE-ROCHE-ST-SAUVES,16.0,SSSAU,-48.0,VOING,-47.0,NaN,NaN,NaN,NaN
1,AUVERGNE-RHÔNE-ALPES,GATEL,GATELLIER,12.0,AURIL,-29.0,JUSSA,-29.0,GATEL,-29.0,NaN,NaN
2,AUVERGNE-RHÔNE-ALPES,ALLE5L31CIZE,LIAISON 63kV N0 1 ALLEMENT (PORTIQUE)-CIZE,1.0,ALLE5,-64.0,PAIRE,-64.0,HAUT6,-46.0,NaN,NaN
3,BOURGOGNE-FRANCHE-COMTÉ,SARRYL61SEREI,LIAISON 225kV N0 1 SARRY-SEREIN,131.0,SEREI,-15.0,AUXER,-14.0,SAUIL,-14.0,NaN,NaN
4,BOURGOGNE-FRANCHE-COMTÉ,GISSEL32VERRE,LIAISON 63kV N0 2 GISSEY-VERREY,29.0,VERRE,37.0,GISSE,-31.0,DARCE,-14.0,NaN,NaN


2. [Energies et Puissances régionales liées au contraintes (vision à février 2024)](https://odre.opendatasoft.com/explore/dataset/energies-et-puissances-regionales-liees-au-contraintes/information/)

In [10]:
path = '/Users/matthiasmolenat/repos/congestion/data/energies-et-puissances-regionales-liees-au-contraintes.csv'
df_contraintes_P = pd.read_csv(path,delimiter=';',index_col=False)
df_contraintes_P.sort_values(by='Puissance Totale à compenser', ascending= True).head(12)

,Région,Puissance ENR installée,Puissance Totale à compenser,Énergie non évacuée moyenne Printemps,Énergie non évacuée moyenne Été,Énergie non évacuée moyenne Automne,Énergie non évacuée moyenne Hiver
4,ÎLE-DE-FRANCE,953.0,0.0,0.0,0.0,0.0,0.0
11,NORMANDIE,9898.0,2.0,136.0,303.0,638.0,0.0
1,BRETAGNE,3672.0,9.0,2.0,7.0,0.0,0.0
8,AUVERGNE-RHÔNE-ALPES,13251.0,83.0,23.0,1049.0,14.0,0.0
10,PAYS DE LA LOIRE,4131.0,124.0,73.0,977.0,77.0,104.0
2,PROVENCE-ALPES-CÔTE D'AZUR,5990.0,147.0,411.0,4284.0,0.0,2.0
6,OCCITANIE,7014.0,237.0,195.0,14353.0,1271.0,462.0
7,CENTRE-VAL DE LOIRE,3664.0,490.0,4877.0,33696.0,6560.0,40500.0
9,BOURGOGNE-FRANCHE-COMTÉ,4985.0,490.0,13754.0,30485.0,5754.0,69533.0
0,GRAND EST,10008.0,498.0,6844.0,17729.0,2211.0,24013.0


## Data métropole

## Visualisation sur carte

In [11]:
data = df_ligne_RT_2023['Geo Shape'].iloc[1]
print(data)

{"coordinates": [[3.1228486356, 45.8837232437], [3.1226769193, 45.8837870635]], "type": "LineString"}


In [13]:
import plotly.graph_objects as go
import json

# Create an empty figure
fig = go.Figure()

# Iterate over each row in the DataFrame
for index, row in df_ligne_RT_2023.iloc[:1000].iterrows():
    # Parse the JSON string in the 'Geo Shape' column
    try:
        geo_data = json.loads(row['Geo Shape'])
    except json.JSONDecodeError:
        print(f"Error parsing JSON for row {index}. Skipping this row.")
        continue

    # Extract coordinates
    if "coordinates" in geo_data and isinstance(geo_data["coordinates"], list):
        longitudes, latitudes = zip(*geo_data["coordinates"])
        
        # Add a trace for this line
        fig.add_trace(go.Scattermapbox(
            mode = "lines",
            # lon = longitudes,
            # lat = latitudes,
            lon = [longitudes[0], longitudes[-1]],
            lat = [latitudes[0], latitudes[-1]],
            name = f"Line {index}",  # You can replace this with a more meaningful name if available
            line = dict(width = 2)
        ))
    else:
        print(f"No valid coordinates found for row {index}. Skipping this row.")

# Update the layout to use Mapbox
fig.update_layout(
    mapbox_style="carto-darkmatter",
    mapbox=dict(
        center=dict(
            lat=latitudes[0],  # Center on the first latitude
            lon=longitudes[0]  # Center on the first longitude
        ),
        zoom=4  # Adjust this value to zoom in or out
    ),
    showlegend=False
)

# Show the figure
fig.show()